# Object Detection @Edge with SageMaker Neo + Pytorch Yolov5
**SageMaker Studio Kernel**: Data Science

In this exercise you'll:
   - Get a pre-trained model: Yolov5
   - Prepare the model to compile it with Neo
   - Compile the model for the target: **X86_64**
   - Get the optimized model and run a simple local test

### install dependencies

In [ ]:
!apt update -y && apt install -y libgl1
!pip install torch==1.7.0 torchvision==0.8.0 opencv-python dlr==1.8.0

## 1) Get a pre-trained model and export it to torchscript
-> SagMaker Neo expectes the model in the traced format

In [ ]:
import os
import urllib.request

if not os.path.isdir('yolov5'):
    !git clone https://github.com/ultralytics/yolov5 && cd yolov5 && git checkout v5.0

if not os.path.exists('yolov5s.pt'):
    urllib.request.urlretrieve('https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5s.pt', 'yolov5s.pt')

In [ ]:
import torch.nn as nn
import torch
import sys
sys.path.insert(0, 'yolov5')
model = torch.load('yolov5s.pt')['model'].float().cpu()

## We need to replace these two activation functions to make it work with TVM.

# SiLU https://arxiv.org/pdf/1606.08415.pdf ----------------------------------------------------------------------------
class SiLU(nn.Module):  # export-friendly version of nn.SiLU()
    @staticmethod
    def forward(x):
        return x * torch.sigmoid(x)

class Hardswish(nn.Module):  # export-friendly version of nn.Hardswish()
    @staticmethod
    def forward(x):
        # return x * F.hardsigmoid(x)  # for torchscript and CoreML
        return x * F.hardtanh(x + 3, 0., 6.) / 6.  # for torchscript, CoreML and ONNX

for k,m in model.named_modules():
    t = type(m)
    layer_name = f"{t.__module__}.{t.__name__}"    
    if layer_name == 'models.common.Conv':  # assign export-friendly activations
        if isinstance(m.act, nn.Hardswish):
            m.act = Hardswish()
        elif isinstance(m.act, nn.SiLU):
            m.act = SiLU()

img_size=640
inp = torch.rand(1,3,img_size,img_size).float().cpu()
model.model[-1].export = True
p = model(inp)
model_trace = torch.jit.trace(model, inp, strict=False)
model_trace.save('model.pth')

## 2) Create a package with the model and upload to S3

In [ ]:
import tarfile
import sagemaker

sagemaker_session = sagemaker.Session()
model_name='yolov5'

with tarfile.open("model.tar.gz", "w:gz") as f:
    f.add("model.pth")
    f.list()

s3_uri = sagemaker_session.upload_data('model.tar.gz', key_prefix=f'{model_name}/model')
print(s3_uri)

## 3) Compile the model with SageMaker Neo (X86_64)

In [ ]:
import time
import boto3
import sagemaker

role = sagemaker.get_execution_role()
sm_client = boto3.client('sagemaker')
compilation_job_name = f'{model_name}-pytorch-{int(time.time()*1000)}'
sm_client.create_compilation_job(
    CompilationJobName=compilation_job_name,
    RoleArn=role,
    InputConfig={
        'S3Uri': s3_uri,
        'DataInputConfig': f'{{"input": [1,3,{img_size},{img_size}]}}',
        'Framework': 'PYTORCH'
    },
    OutputConfig={
        'S3OutputLocation': f's3://{sagemaker_session.default_bucket()}/{model_name}-pytorch/optimized/',
        'TargetPlatform': { 
            'Os': 'LINUX', 
            'Arch': 'X86_64',
        },
    },
    StoppingCondition={ 'MaxRuntimeInSeconds': 900 }
)
while True:
    resp = sm_client.describe_compilation_job(CompilationJobName=compilation_job_name)    
    if resp['CompilationJobStatus'] in ['STARTING', 'INPROGRESS']:
        print('Running...')
    else:
        print(resp['CompilationJobStatus'], compilation_job_name)
        break
    time.sleep(5)

## 4) Download the compiled model

In [ ]:
output_model_path = f's3://{sagemaker_session.default_bucket()}/{model_name}-pytorch/optimized/model-LINUX_X86_64.tar.gz'
!aws s3 cp $output_model_path /tmp/model.tar.gz
!rm -rf model_object_detection && mkdir model_object_detection
!tar -xzvf /tmp/model.tar.gz -C model_object_detection

## 5) Run the model locally

In [ ]:
import urllib.request
urllib.request.urlretrieve('https://i2.wp.com/petcaramelo.com/wp-content/uploads/2020/05/doberman-cores.jpg?resize=608%2C405', 'dogs.jpg')

In [ ]:
%matplotlib inline
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

# Classes
labels= ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
        'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
        'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
        'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
        'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
        'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
        'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
        'hair drier', 'toothbrush']  # class names

### load the model using the runtime DLR

In [ ]:
import dlr
# load the model (CPU x86_64)
model = dlr.DLRModel('model_object_detection', 'cpu')

In [ ]:
import sys
sys.path.insert(0,'../models/01_YoloV5/01_Pytorch')
from processing import Processor
proc = Processor(labels, threshold=0.25, iou_threshold=0.45)
img = cv2.imread('dogs.jpg')
x = proc.pre_process(img)
y = model.run(x)
(bboxes, scores, cids), image = proc.post_process(y, img.shape, img.copy())
plt.figure(figsize=(10,10))
plt.imshow(image)

# Done! :)